## Welcome to Quantum Computing!

As a full stack developer with a physics background, I'm excited to share my journey into quantum computing. This post will guide you through creating your first quantum circuit using Qiskit.

### What You'll Learn

- Setting up Qiskit in your environment
- Understanding qubits vs classical bits
- Creating and visualizing quantum circuits
- Running quantum simulations

## Prerequisites

```bash
pip install qiskit
pip install qiskit-aer
pip install matplotlib
```

## Step 1: Import Qiskit

Let's start by importing the necessary libraries:

In [ ]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit_aer import Aer
from qiskit.visualization import plot_histogram, circuit_drawer
import matplotlib.pyplot as plt

# Set matplotlib to inline for Jupyter
%matplotlib inline

print("✅ Qiskit imported successfully!")

## Step 2: Create Your First Quantum Circuit

A quantum circuit is like a classical circuit, but instead of bits (0 or 1), we use **qubits** that can exist in superposition.

Let's create a simple circuit with 2 qubits:

In [ ]:
# Create a quantum circuit with 2 qubits and 2 classical bits
qc = QuantumCircuit(2, 2)

# Apply a Hadamard gate to qubit 0 (creates superposition)
qc.h(0)

# Apply a CNOT gate (creates entanglement)
qc.cx(0, 1)

# Measure both qubits
qc.measure([0, 1], [0, 1])

# Draw the circuit
qc.draw('mpl')

### What's Happening Here?

1. **Hadamard Gate (H)**: Puts qubit 0 into superposition
   - $|0⟩ \rightarrow \frac{1}{\sqrt{2}}(|0⟩ + |1⟩)$
   
2. **CNOT Gate**: Creates entanglement between qubits
   - If qubit 0 is |1⟩, flip qubit 1
   
3. **Measurement**: Collapses the quantum state to classical bits

This circuit creates a **Bell State** - a maximally entangled quantum state!

## Step 3: Simulate the Circuit

Let's run our circuit on a quantum simulator:

In [ ]:
# Use Aer's qasm simulator
simulator = Aer.get_backend('qasm_simulator')

# Execute the circuit 1000 times
job = simulator.run(qc, shots=1000)
result = job.result()

# Get the counts
counts = result.get_counts(qc)
print("Measurement results:", counts)

# Plot the histogram
plot_histogram(counts)

## What Do The Results Mean?

You should see approximately 50% **|00⟩** and 50% **|11⟩**.

This is **quantum entanglement** in action:
- Both qubits are always measured in the same state
- We never see |01⟩ or |10⟩
- This correlation exists even though each qubit is in superposition

### Classical vs Quantum

With classical bits, you'd need to communicate between them to correlate their states. With quantum entanglement, the correlation is intrinsic to the system.

## Next Steps

Now that you've created your first quantum circuit, try:

1. **Modify the circuit** - Add more gates
2. **Explore other gates** - X, Y, Z, S, T gates
3. **Run on real quantum hardware** - Use IBM Quantum's free tier

## Resources

- [Qiskit Documentation](https://qiskit.org/documentation/)
- [IBM Quantum Lab](https://quantum-computing.ibm.com/)
- [Qiskit Textbook](https://qiskit.org/learn)

---

**Questions?** Drop a comment or reach out on [GitHub](https://github.com/dcoldeira)!